In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
img_size = 48
batch_size = 32
epochs = 20
learning_rate = 3e-4
step_size = 5
gamma = 0.1
data_dir = '/content/drive/MyDrive/dataset_gender'
save_path = '/content/drive/MyDrive/gender.pth'

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Computed RGB mean: [0.5728868246078491, 0.5101767778396606, 0.47214433550834656], std: [0.29351043701171875, 0.2858504056930542, 0.2939562499523163]


In [ ]:
rgb_mean = [0.485, 0.456, 0.406]
rgb_std = [0.229, 0.224, 0.225]

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


In [ ]:
train_tf = transforms.Compose([
    transforms.RandomResizedCrop(img_size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(rgb_mean, rgb_std),
])
test_tf = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.CenterCrop(img_size),
    transforms.ToTensor(),
    transforms.Normalize(rgb_mean, rgb_std),
])

Classes: ['Female', 'Male']


In [ ]:
# data
train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=train_tf)
test_dataset = datasets.ImageFolder(os.path.join(data_dir, 'test'), transform=test_tf)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

classes = train_dataset.classes
num_classes = len(classes)

In [ ]:
# model
import timm
model = timm.create_model(
    'mobilevit_s',
    pretrained=True,
    num_classes=num_classes,
    in_chans=3
).to(device)

In [ ]:
# Loss, Optimizer, Scheduler
import torch.nn as nn
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-2)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

# train
best_acc = 0.0
for epoch in range(1, epochs + 1):
    model.train()
    running_loss = 0.0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)
    scheduler.step()
    epoch_loss = running_loss / len(train_dataset)

    model.eval()
    correct = total = 0
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    test_acc = correct / total * 100
    print(f"Epoch {epoch:02d}: Train Loss={epoch_loss:.4f}, Test Acc={test_acc:.2f}%")
    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), save_path)
        print(f" New best model saved with acc={best_acc:.2f}%")
print(f"Best Test Accuracy: {best_acc:.2f}%")

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 01: Train Loss=0.5315, Test Acc=75.68%
 New best model saved with acc=75.68%
Epoch 02: Train Loss=0.4436, Test Acc=78.84%
 New best model saved with acc=78.84%
Epoch 03: Train Loss=0.4137, Test Acc=76.71%
Epoch 04: Train Loss=0.3963, Test Acc=79.43%
 New best model saved with acc=79.43%
Epoch 05: Train Loss=0.3790, Test Acc=81.94%
 New best model saved with acc=81.94%
Epoch 06: Train Loss=0.3459, Test Acc=83.33%
 New best model saved with acc=83.33%
Epoch 07: Train Loss=0.3423, Test Acc=82.73%
Epoch 08: Train Loss=0.3337, Test Acc=83.13%
Epoch 09: Train Loss=0.3321, Test Acc=83.62%
 New best model saved with acc=83.62%
Epoch 10: Train Loss=0.3263, Test Acc=83.52%
Epoch 11: Train Loss=0.3277, Test Acc=83.67%
 New best model saved with acc=83.67%
Epoch 12: Train Loss=0.3172, Test Acc=83.72%
 New best model saved with acc=83.72%
Epoch 13: Train Loss=0.3328, Test Acc=83.18%
Epoch 14: Train Loss=0.3175, Test Acc=83.62%
Epoch 15: Train Loss=0.3199, Test Acc=83.77%
 New best model saved

In [ ]:
# predict
from PIL import Image
model.load_state_dict(torch.load(save_path, map_location=device))
model.eval()

def predict(img_path):
    img = Image.open(img_path).convert('RGB')
    x = test_tf(img).unsqueeze(0).to(device)
    with torch.no_grad():
        logits = model(x)
    idx = logits.argmax(dim=1).item()
    return classes[idx]

# pred = predict('/content/drive/MyDrive/archive/test/happy/xxx.jpg')
# print('Predicted gender:', pred)